In [14]:
import pyscheduling.JS.JmCmax as jm_cmax
import pyscheduling.JS.JmriwiTi as jm_wiTi
from pyscheduling.JS.JobShop import Job, JobShopSolution
from pyscheduling.Problem import Objective

import gurobipy as gp
from gurobipy import GRB, quicksum, multidict, tuplelist, tupledict

import networkx as nx

%load_ext autoreload
%load_ext pycodestyle_magic
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [2]:
def transform_solution(Y_ij, instance, dict_m_jobs, dict_ij):
    
    sol = JobShopSolution(instance)
    
    for m in dict_m_jobs:
        jobs_list = []
        for j in dict_m_jobs[m]:
            job_start = int(Y_ij[m, j].x)
            jobs_list.append(Job(j , job_start, job_start + dict_ij[(m,j)]))
        
        sol.machines[m].job_schedule = sorted(jobs_list, key= lambda x: x[1])
    
    return sol

In [3]:
instance = jm_cmax.JmCmax_Instance.generate_random(8, 3)
E = range(instance.n)
M = range(instance.m)

jobs_prec = { job_id: [ (instance.P[job_id][i-1][0], instance.P[job_id][i][0]) \
                       for i in range(1, len(instance.P[job_id])) ] for job_id in E}

dict_ij = [ {(element[0],job_id): element[1] for element in job_l } \
               for job_id, job_l in enumerate(instance.P)  ]
dict_ij = {element: element_list[element] for element_list in dict_ij for element in element_list}

dict_m_jobs = {}
for m in M:
    dict_m_jobs[m] = [j for (i,j) in dict_ij if i == m]

model = gp.Model("jobshop")

Y_ij = model.addVars(dict_ij, vtype=gp.GRB.INTEGER, lb=0, name="Y")
C_max = model.addVar(lb=0, vtype=gp.GRB.INTEGER, name="C_max")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-15


In [5]:
instance.P

[[(1, 53), (2, 72)],
 [(1, 33), (0, 31)],
 [(2, 71), (1, 17)],
 [(1, 27), (0, 89)],
 [(2, 45)],
 [(2, 46), (1, 62)],
 [(2, 43)],
 [(0, 77)]]

In [6]:
dict_m_jobs

{0: [1, 3, 7], 1: [0, 1, 2, 3, 5], 2: [0, 2, 4, 5, 6]}

In [7]:
# Adding the constraints
model.addConstrs( (Y_ij[k,j] - Y_ij[i, j] >= dict_ij[(i,j)] for j in E for (i, k) in jobs_prec[j]), name="prec")
model.addConstrs( (C_max - Y_ij[i,j] >= dict_ij[(i,j)] for (i, j) in dict_ij), name="cmax")
model.addConstrs( (Y_ij[m, dict_m_jobs[m][i-1]] - Y_ij[m, dict_m_jobs[m][i]] >= dict_ij[(m, dict_m_jobs[m][i])] \
                   for m in M for i in range(1,len(dict_m_jobs[m])) ) , name="machine")

# Setting the objective function
model.setObjective( C_max ) # Cmax

# Tuning and Calling the solver 
#model.write("model.lp")
model.update()
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 28 rows, 14 columns and 56 nonzeros
Model fingerprint: 0x3dc58d96
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 9e+01]
Found heuristic solution: objective 380.0000000
Presolve removed 28 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 380 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.800000000000e+02, best bound 3.800000000000e+02, gap 0.0000%


In [8]:
sol = transform_solution(Y_ij, instance, dict_m_jobs, dict_ij)

In [10]:
sol.machines

[Machine(machine_num=0, objective=0, last_job=-1, job_schedule=[Job(id=7, start_time=0, end_time=77), Job(id=3, start_time=178, end_time=267), Job(id=1, start_time=267, end_time=298)]),
 Machine(machine_num=1, objective=0, last_job=-1, job_schedule=[Job(id=5, start_time=89, end_time=151), Job(id=3, start_time=151, end_time=178), Job(id=2, start_time=205, end_time=222), Job(id=1, start_time=222, end_time=255), Job(id=0, start_time=255, end_time=308)]),
 Machine(machine_num=2, objective=0, last_job=-1, job_schedule=[Job(id=6, start_time=0, end_time=43), Job(id=5, start_time=43, end_time=89), Job(id=4, start_time=89, end_time=134), Job(id=2, start_time=134, end_time=205), Job(id=0, start_time=308, end_time=380)])]

In [19]:
sol = jm_cmax.Heuristics.shifting_bottleneck(instance)

IndexError: pop from empty list